In [1]:
import pandas as pd

In [4]:
import glob
DATA_PATH = '../data/'
DATA_PROCESSED_PATH = DATA_PATH + 'processed/'
filepath = DATA_PROCESSED_PATH + '*.csv'
# Get training and test data
train_path = glob.glob(DATA_PROCESSED_PATH + 'train*.csv')[0]
val_path = glob.glob(DATA_PROCESSED_PATH + 'val*.csv')[0]

In [6]:
import sys
sys.path.append('../src')
from models.models import calculate_baseline, run_models
from models.pipeline import make_features_pipeline

# Load training and validation data
train = pd.read_csv(train_path)
val = pd.read_csv(val_path)

# Train test split
X_train = train.drop('hyperpartisan', axis=1)
y_train = train['hyperpartisan']
X_test = val.drop('hyperpartisan', axis=1)
y_test = val['hyperpartisan']

# Calculate Baseline
baseline = calculate_baseline(train)

# Create Feature Union
feats = make_features_pipeline()

# Evaluate Models
model_list = ['nb', 'lr']
best_tfidf_model, best_tfidf_model_type, best_tfidf_model_predictions = run_models(feats, model_list, X_train, X_test, y_train, y_test)

Majority baseline accuracy is 0.4999
Multinomial Naive Bayes
             precision    recall  f1-score   support

      False       0.77      0.38      0.51      1249
       True       0.59      0.89      0.71      1250

avg / total       0.68      0.63      0.61      2499

Accuracy: 0.6339
Logistic Regression
             precision    recall  f1-score   support

      False       0.71      0.48      0.57      1249
       True       0.61      0.80      0.69      1250

avg / total       0.66      0.64      0.63      2499

Accuracy: 0.6403
Best model is Logistic Regression with an accuracy score of 0.6403


In [10]:
preds = pd.Series(best_tfidf_model_predictions)

In [20]:
pvt = pd.DataFrame({'true': y_test, 'pred' : preds})

In [22]:
pvt[(pvt['true']==True) & (pvt['pred']==False)].index

Int64Index([ 504,  507,  512,  517,  520,  521,  523,  524,  526,  533,
            ...
            2161, 2170, 2172, 2177, 2178, 2181, 2185, 2199, 2201, 2204],
           dtype='int64', length=245)

In [29]:
val.iloc[pvt[(pvt['true']==True) & (pvt['pred']==False)].index].reset_index()['url'][201]

'https://pjmedia.com/parenting/2015/11/30/top-ten-christmas-movies-we-love-to-watch-over-and-over-again'